In [1]:
import json
import gc
import random
import networkx as nx
from networkx.readwrite import json_graph
import numpy as np
import pandas as pd
from tqdm import tqdm
from typing import Dict, List
import scml
import mylib

In [2]:
# number of context words in the left and right directions
m = 4
iters = 1
limit = 0
max_choices = 1000
type2id = {
    "click": 0,
    "cart": 1,
    "order": 2,
}

In [3]:
tim = scml.Timer()
tim.start()
percentiles=[.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]
pd.set_option("use_inf_as_na", True)
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()
scml.seed_everything(31)

In [4]:
%%time
with open("input/vocab3.json") as f:
    id2label = [w for w, c in json.load(f)]
label2id = {k:v for v, k in enumerate(id2label)}
print(f"len(label2id)={len(label2id):,}")

len(label2id)=1,855,603
Wall time: 2.41 s


In [5]:
del id2label
gc.collect()

21

In [6]:
%%time
with open("input/graph.json") as f:
    g = nx.adjacency_graph(json.load(f))
print(g)

DiGraph with 3764159 nodes and 100953434 edges
Wall time: 9min 5s


In [7]:
rows = []
_nodes = g.nodes
if limit>0:
    _nodes = list(_nodes)[:limit]
out_edges = {}
in_edges = {}
for i in range(iters):
    for center in tqdm(_nodes, desc=f"i={i}"):
        # going left: predecessor nodes
        center_word, center_type = tuple(center.split("_"))
        curr = center
        for _ in range(m):
            if curr not in in_edges:
                choices = []
                for u, v, d in g.in_edges(curr, data=True):
                    if len(choices)>max_choices:
                        break
                    choices.append((u, d["weight"]))
                in_edges[curr] = choices
            choices = in_edges[curr]
            if len(choices)==0:
                break
            curr = mylib.weighted_choice(choices)
            if curr==center:
                break
            curr_word, curr_type = tuple(curr.split("_"))
            if curr_word!=center_word:
                rows.append({
                    "center_word": label2id[center_word],
                    "center_type": type2id[center_type],
                    "outside_word": label2id[curr_word],
                    "outside_type": type2id[curr_type],
                })
        # going right: successor nodes
        curr = center
        for _ in range(m):
            if curr not in out_edges:
                choices = []
                for u, v, d in g.out_edges(curr, data=True):
                    if len(choices)>max_choices:
                        break
                    choices.append((v, d["weight"]))
                out_edges[curr] = choices
            choices = out_edges[curr]
            if len(choices)==0:
                break
            curr = mylib.weighted_choice(choices)
            if curr==center:
                break
            curr_word, curr_type = tuple(curr.split("_"))
            if curr_word!=center_word:
                rows.append({
                    "center_word": label2id[center_word],
                    "center_type": type2id[center_type],
                    "outside_word": label2id[curr_word],
                    "outside_type": type2id[curr_type],
                })

i=0: 100%|███████████████████| 3764159/3764159 [41:40<00:00, 1505.07it/s]


In [8]:
df = pd.DataFrame.from_records(rows)
df.drop_duplicates(keep="first", inplace=True, ignore_index=True)
cols = ["center_word", "outside_word"]
df[cols] = df[cols].astype(np.int32)
cols = ["center_type", "outside_type"]
df[cols] = df[cols].astype(np.int8)
df.info()
assert (df["center_word"]!=df["outside_word"]).all()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26703700 entries, 0 to 26703699
Data columns (total 4 columns):
 #   Column        Dtype
---  ------        -----
 0   center_word   int32
 1   center_type   int8 
 2   outside_word  int32
 3   outside_type  int8 
dtypes: int32(2), int8(2)
memory usage: 254.7 MB


In [9]:
df.describe(percentiles=percentiles)

,center_word,center_type,outside_word,outside_type
count,2.670370e+07,2.670370e+07,2.670370e+07,2.670370e+07
mean,7.944473e+05,6.451010e-01,3.146382e+05,2.147599e-01
std,5.190362e+05,7.517711e-01,4.174689e+05,5.465662e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1%,1.324600e+04,0.000000e+00,6.800000e+01,0.000000e+00
5%,6.568300e+04,0.000000e+00,8.150000e+02,0.000000e+00
10%,1.321000e+05,0.000000e+00,3.079000e+03,0.000000e+00
20%,2.693790e+05,0.000000e+00,1.324400e+04,0.000000e+00
30%,4.146130e+05,0.000000e+00,3.328000e+04,0.000000e+00
40%,5.697950e+05,0.000000e+00,6.748000e+04,0.000000e+00


In [10]:
df.head(30)

,center_word,center_type,outside_word,outside_type
0,1,1,103,0
1,35,0,55,1
2,35,0,55,0
3,35,0,703,0
4,35,0,831,0
5,7,0,41,0
6,7,0,259,0
7,7,0,445,0
8,7,0,653,0
9,7,0,1534,0


In [11]:
%%time
assert df.notna().all(axis=None)
df.to_parquet("output/pairs.parquet", index=False)

Wall time: 1.85 s


In [12]:
tim.stop()
print(f"Total time taken {str(tim.elapsed)}")

Total time taken 0:51:39.449719
